In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df_train = pd.read_csv("../input/train.csv")

In [ ]:
df_test = pd.read_csv("../input/test.csv")

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.head()

In [ ]:
#check the columns
df_train.columns

## Missing data

In [ ]:
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(30)

## Null Value Treatment

In [ ]:
def nullvalue_function(df_train,percentage):
    
    # Checking the null value occurance
    
    print(df_train.isna().sum())

    # Printing the shape of the data 
    
    print(df_train.shape)
    
    # Converting  into percentage table
    
    null_value_table=pd.DataFrame((df_train.isna().sum()/df_train.shape[0])*100).sort_values(0,ascending=False )
    
    null_value_table.columns=['null percentage']
    
    # Defining the threashold values 
    
    null_value_table[null_value_table['null percentage']>percentage].index
    
    # Drop the columns that has null values more than threashold 
    df_train.drop(null_value_table[null_value_table['null percentage']>percentage].index,axis=1,inplace=True)
    
    # Replace the null values with median() # continous variables 
    for i in df_train.describe().columns:
        df_train[i].fillna(df_train[i].median(),inplace=True)
    # Replace the null values with mode() #categorical variables
    for i in df_train.describe(include='object').columns:
        df_train[i].fillna(df_train[i].value_counts().index[0],inplace=True)
  
    print(df_train.shape)
    
    return df_train

In [ ]:
df_train=nullvalue_function(df_train,50)

In [ ]:
## Top 20 columns important as per variance
df_train.var().sort_values(ascending=False).index[0:20]

In [ ]:
## Continuous and categorical columns

cont=df_train.describe().columns
cat=df_train.describe(include='object').columns

In [ ]:
cat

In [ ]:
cat.shape

In [ ]:
cont

In [ ]:
cont.shape

In [ ]:
df_train_cat=df_train[cat]

In [ ]:
df_train_cat.shape

In [ ]:
for i in df_train_cat.columns:
    print(i,df_train_cat[i].nunique())

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
for i in df_train_cat.columns:
    le = preprocessing.LabelEncoder()
    le.fit(df_train_cat[i])
    transformed=le.transform(df_train_cat[i]) 
    df_train_cat[i]=transformed
    

In [ ]:
df_train_cat.columns

In [ ]:
df_train_cont=df_train[cont]

In [ ]:
df_train_cont.shape

In [ ]:
df_train_cat.shape

In [ ]:
df_train.shape

In [ ]:
df_train_cont.head()

In [ ]:
df_train_cat.head()

In [ ]:
analytical_dataset=pd.DataFrame()

In [ ]:
for i in df_train_cont.columns:
    analytical_dataset[i]=df_train_cont[i]

In [ ]:
for i in df_train_cat.columns:
    analytical_dataset[i]=df_train_cat[i]

In [ ]:
analytical_dataset.shape

In [ ]:
## Prioritizing the variables based on varaince 

analytical_dataset_var=analytical_dataset[analytical_dataset.var().sort_values(ascending=False).index[0:30]]

In [ ]:
analytical_dataset_var.head()

## Normalization of the dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(analytical_dataset_var)
normalized_x=scaler.transform(analytical_dataset_var)

In [ ]:
analytica_dataset_norm=pd.DataFrame(normalized_x)

In [ ]:
analytica_dataset_norm.columns=analytical_dataset_var.columns

In [ ]:
analytica_dataset_norm.head()

In [ ]:
analytica_dataset_norm.shape

In [ ]:
analytica_dataset_norm['SalePrice']=df_train['SalePrice']

In [ ]:
#descriptive statistics summary
analytica_dataset_norm['SalePrice'].describe()

In [ ]:
#histogram
sns.distplot(analytica_dataset_norm['SalePrice']);

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % analytica_dataset_norm['SalePrice'].skew())
print("Kurtosis: %f" % analytica_dataset_norm['SalePrice'].kurt())

In [ ]:
analytica_dataset_norm.columns

In [ ]:
#scatterplot
col=['SalePrice', 'LotArea', 'GrLivArea', 'MiscVal', 'BsmtFinSF1','BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF',]
sns.pairplot(analytica_dataset_norm[col], size = 2.5)
plt.show();

In [ ]:
plt.figure(figsize=(10,10))
col=['SalePrice', 'LotArea', 'GrLivArea', 'MiscVal', 'BsmtFinSF1','BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF',]
sns.heatmap(analytica_dataset_norm[col].corr(),annot=True)

## Model Building

In [ ]:
y=analytica_dataset_norm['SalePrice']
x=analytica_dataset_norm.drop('SalePrice',axis=1)

In [ ]:
df_test.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
print(x.shape,y.shape,x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
### KNN Algorithms

from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor(n_neighbors=5)
knn.fit(x_train,y_train)
predicted=knn.predict(x_test)

In [ ]:
predicted

In [ ]:
y_test.values

In [ ]:
pd.DataFrame(y_test.values,predicted).head()

In [ ]:
data_comparision=pd.DataFrame(y_test.values,predicted)
data_comparision.reset_index(inplace=True)

In [ ]:
data_comparision.columns=['actual','predicted']

In [ ]:
data_comparision.head()

In [ ]:
data_comparision[0:292].plot(figsize=(20,5))

In [ ]:
Error=abs(sum(data_comparision['actual']-data_comparision['predicted']))

In [ ]:
Error